#**Computer Vision CO468** 
##**Assignment-2 Results**
TEAM

Shushant Kumar - 16CO143

Anmol Horo     - 16CO206

Edwin Thomas   - 16CO218

IMPORTS


In [0]:
import numpy as np
import cv2
from imblearn.metrics import sensitivity_specificity_support
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn import metrics

LOADING TEST DATA


In [0]:
# Loading the dataset
test_x = np.load("/path/to/testx.npy")    # Kindly add the path to test_x (images)
test_y = np.load("/path/to/testy.npy")    # Kindly add the path to test_y (labels)

print(test_x.shape)
print(test_y.shape)

(587, 512, 512, 3)
(587,)


NORMALIZATION


In [0]:
# Normalization
mean_t = np.mean(test_x)
std_t = np.std(test_x)
test_x = test_x - mean_t
test_x = test_x/std_t

MODEL ARCHITECTURE

In [6]:
## Model 1 Architecture
import tensorflow as tf
from keras import backend
from keras import models
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
import os
from keras import Model
from keras.applications import imagenet_utils
from keras import layers
from keras.applications.imagenet_utils import decode_predictions

def dense_layer(y, blocks, name):
    for i in range(blocks):
        y = conv_layer(y, 32, name=name + '_block' + str(i + 1))
    return y


def transition_layer(y, reduction, name):
    bn_axis = 3 
    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(y)
    y = layers.Activation('relu', name=name + '_relu')(y)
    y = layers.Conv2D(int(backend.int_shape(y)[bn_axis] * reduction), 1, use_bias=False, name=name + '_conv')(y)
    y = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(y)
    return y


def conv_layer(y, growth_rate, name):
    bn_axis = 3 
    y1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(y)
    y1 = layers.Activation('relu', name=name + '_0_relu')(y1)
    y1 = layers.SeparableConv2D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(y1)
    y1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(y1)
    y1 = layers.Activation('relu', name=name + '_1_relu')(y1)
    y1 = layers.SeparableConv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(y1)
    y = layers.Concatenate(axis=bn_axis, name=name + '_concatenate')([y, y1])
    return y

def Model1():
    bn_axis = 3

    image=layers.Input((512,512,3))
    y = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(image)
    y = layers.Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(y)
    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1/bn')(y)
    y = layers.Activation('relu', name='conv1/relu')(y)
    y = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(y)
    y = layers.MaxPooling2D(3, strides=2, name='pool1')(y)

    y = dense_layer(y, 6, name='conv2')
    y = transition_layer(y, 0.5, name='pool2')
    y = dense_layer(y, 12, name='conv3')
    y = transition_layer(y, 0.5, name='pool3')
    y = dense_layer(y, 24, name='conv4')
    y = transition_layer(y, 0.5, name='pool4')
    y = dense_layer(y, 16, name='conv5')

    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(y)
    y = layers.Activation('relu', name='relu')(y)
    y2 = GlobalAveragePooling2D()(y)
    y3 = Dropout(0.3)(y2)
    output1 = Dense(10,activation='relu')(y3)
    output2 = Dense(3,activation='softmax')(output1)

    model = models.Model( image, output2, name='model_1')
    return model

from keras.optimizers import Adam
model=Model1()
model.compile(optimizer=Adam(lr=3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

LOADING THE TRAINED MODEL

In [0]:
# Loading the saved weights
model.load_weights('weights/model1_best_normalized.h5')  # Kindly add the path to the weights file here
# The weight file associated with this model is model1_best_normalized.h5

PREDICTION ON TEST DATA

In [0]:
# Prediction on the test set 
test_pred = model.predict(test_x, batch_size=8)

pred_y = np.zeros(len(test_pred))
for i in range(len(test_pred)):
  if test_pred[i][0] >= test_pred[i][1] and test_pred[i][0] >= test_pred[i][2]:
    pred_y[i] = 0
  elif test_pred[i][1] >= test_pred[i][0] and test_pred[i][1] >= test_pred[i][2]:
    pred_y[i] = 1
  else:
    pred_y[i] = 2

#Results Section

CONFUSION MATRIX




In [9]:
print(metrics.confusion_matrix(test_y, pred_y))

[[ 45   0   4]
 [  0 266   3]
 [  1  13 255]]


CLASSIFICATION REPORT

In [10]:
print(metrics.classification_report(test_y, pred_y, digits=3))

              precision    recall  f1-score   support

           0      0.978     0.918     0.947        49
           1      0.953     0.989     0.971       269
           2      0.973     0.948     0.960       269

    accuracy                          0.964       587
   macro avg      0.968     0.952     0.960       587
weighted avg      0.965     0.964     0.964       587



F1 SCORE

In [19]:
print('F1 Score:', f1_score(test_y,pred_y , average="macro"))

F1 Score: 0.9595411060539302


PRECISION


In [18]:
print('Precision:', precision_score(test_y,pred_y , average="macro"))

Precision: 0.968316110078152


SENSITIVITY

In [13]:
print('Sensitivity:', sensitivity_specificity_support(test_y,pred_y , average='macro')[0])

Sensitivity: 0.9517234403054902


SPECIFICITY

In [14]:
print('Specificity:', sensitivity_specificity_support(test_y,pred_y , average='macro')[1])

Specificity: 0.9784160607265048


RECALL

In [15]:
print('Recall:', recall_score(test_y,pred_y, average='macro'))

Recall: 0.9517234403054902


ACCURACY

In [16]:
print('Accuracy:' ,accuracy_score(test_y,pred_y ))

Accuracy: 0.9642248722316865
